<a href="https://colab.research.google.com/github/drwitt/BME_590_Tensorflow_Deep_Learning/blob/master/BME_MML_Lecture_2_Tensorflow_Fundamentals_and_Python_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Dependencies

In [0]:
!pip3 install tensorflow_datasets
!pip3 install tensorflow-gpu==2.0.0-rc0

%load_ext tensorboard

!nvidia-smi

# Review decorators and with blocks

In [0]:
class controlled_execution:
    def __enter__(self):
        print('setting things up (such as opening a file)')
    def __exit__(self, type, value, traceback): # What are the __underscores__ for?
        print(value)
        print(traceback)
        print(type)
        print('removing things (such as closing a file)')

with controlled_execution():
  x = 1/0

In [0]:
def my_decorator(fn):
  def function_wrapper(x):
    print("Before calling " + fn.__name__)
    output = fn(x)
    print("After calling " + fn.__name__)
    return output
  return function_wrapper

@my_decorator
def my_fun(x):
  return x+x

a = my_fun(2)
print(a)

# Explore `@tf.function`
We will create the following graph in TensorFlow 2.0 using `@tf.function`

![alt text](https://miro.medium.com/max/1917/1*RfSvkiVUHHWEe805Hf1HEw.png)

In [0]:
import tensorflow as tf

@tf.function
def my_func(my_input):
    a = tf.square(my_input, name="A")
    b = tf.cos(a, name="B")
    c = tf.sin(a, name="C")   
    d = tf.add(b, c, name="D")
    e = tf.floor(b, name="E")
    f = tf.sqrt(d, name="F")
    return e,f

# We use `tf.summary` to trace the graph as it is executing.
tf.summary.trace_on(graph=True, profiler=False)
print(my_func(2.0))
with tf.summary.create_file_writer('./logs').as_default():   
    tf.summary.trace_export(
        name='tf2_graph',
        step=0,
        profiler_outdir='./logs/profiler'
    )
tf.summary.trace_off()

(<tf.Tensor: id=301, shape=(), dtype=float32, numpy=-1.0>, <tf.Tensor: id=302, shape=(), dtype=float32, numpy=nan>)


In [0]:
# We can also get an explicit tf.Graph using this with block
graph = tf.Graph()
with graph.as_default():
  my_func(2.0)
graph.as_graph_def()

In [0]:
%tensorboard --logdir='./logs'

Reusing TensorBoard on port 6006 (pid 224), started 0:43:26 ago. (Use '!kill 224' to kill it.)

In [0]:
# tf.Variable
# @tf.function (what happens if we try this?)
def f(const):
    a = tf.constant([[10,10],[11.,1.]])
    x = tf.constant([[1.,0.],[0.,1.]])
    b = tf.Variable(12.) + const
    y = tf.matmul(a, x) + b
    print("PRINT: ", y)
    return y

f(100)

PRINT:  tf.Tensor(
[[122. 122.]
 [123. 113.]], shape=(2, 2), dtype=float32)


<tf.Tensor: id=438, shape=(2, 2), dtype=float32, numpy=
array([[122., 122.],
       [123., 113.]], dtype=float32)>

In [0]:
# What is the difference between class and instance?

class F:
    def __init__(self):
        self._b = None

    @tf.function
    def __call__(self, const):
        a = tf.constant([[10, 10], [11., 1.]])
        x = tf.constant([[1., 0.], [0., 1.]])
        if self._b is None:
            self._b = tf.Variable(12.)
        self._b.assign(self._b + const)
        y = tf.matmul(a, x) + self._b
        print("PRINT: ", y)
        tf.print("TF-PRINT: ", y)
        return y

f = F()
f(100.)

PRINT:  Tensor("add_1:0", shape=(2, 2), dtype=float32)
PRINT:  Tensor("add_1:0", shape=(2, 2), dtype=float32)
TF-PRINT:  [[122 122]
 [123 113]]


<tf.Tensor: id=623, shape=(2, 2), dtype=float32, numpy=
array([[122., 122.],
       [123., 113.]], dtype=float32)>